<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (SparkSession
  .builder
  .appName("App")
  .getOrCreate())

24/06/19 21:37:58 WARN Utils: Your hostname, MacBook-Pro-Alexey.local resolves to a loopback address: 127.0.0.1; using 192.168.1.138 instead (on interface en0)
24/06/19 21:37:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/19 21:37:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
parsed = spark.read.option('header', 'true').option('nullValue', '?').\
         option('inferShema', 'true').csv('/Users/alexfil/linkage/block*.csv')

In [4]:
parsed.first()

Row(id_1='3148', id_2='8326', cmp_fname_c1='1', cmp_fname_c2=None, cmp_lname_c1='1', cmp_lname_c2=None, cmp_sex='1', cmp_bd='1', cmp_bm='1', cmp_by='1', cmp_plz='1', is_match='TRUE')

In [5]:
parsed.cache()

DataFrame[id_1: string, id_2: string, cmp_fname_c1: string, cmp_fname_c2: string, cmp_lname_c1: string, cmp_lname_c2: string, cmp_sex: string, cmp_bd: string, cmp_bm: string, cmp_by: string, cmp_plz: string, is_match: string]

In [6]:
parsed.show(2)

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| 3148| 8326|           1|        NULL|           1|        NULL|      1|     1|     1|     1|      1|    TRUE|
|14055|94934|           1|        NULL|           1|        NULL|      1|     1|     1|     1|      1|    TRUE|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
only showing top 2 rows



In [7]:
parsed.cache()

24/06/19 21:38:36 WARN CacheManager: Asked to cache already cached data.


DataFrame[id_1: string, id_2: string, cmp_fname_c1: string, cmp_fname_c2: string, cmp_lname_c1: string, cmp_lname_c2: string, cmp_sex: string, cmp_bd: string, cmp_bm: string, cmp_by: string, cmp_plz: string, is_match: string]

In [8]:
from pyspark.sql.functions import col

In [9]:
parsed.groupBy('is_match').count().orderBy(col('Count').desc()).show()

+--------+-------+
|is_match|  count|
+--------+-------+
|   FALSE|5728201|
|    TRUE|  20931|
+--------+-------+



In [10]:
from pyspark.sql.functions import avg, stddev

In [11]:
parsed.agg(avg('cmp_sex'), stddev('cmp_sex')).show()

+-----------------+------------------+
|     avg(cmp_sex)|   stddev(cmp_sex)|
+-----------------+------------------+
|0.955001381078048|0.2073011111689795|
+-----------------+------------------+



In [12]:
parsed.createOrReplaceTempView('linkage')

In [13]:
spark.sql("""
    SELECT is_match, COUNT(*) cnt
    FROM linkage
    GROUP BY is_match
    ORDER BY cnt DESC
""").show()

+--------+-------+
|is_match|    cnt|
+--------+-------+
|   FALSE|5728201|
|    TRUE|  20931|
+--------+-------+



In [14]:
summary = parsed.describe()
summary.show()

24/06/19 21:38:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+--------+
|summary|              id_1|              id_2|      cmp_fname_c1|      cmp_fname_c2|      cmp_lname_c1|       cmp_lname_c2|           cmp_sex|             cmp_bd|             cmp_bm|             cmp_by|            cmp_plz|is_match|
+-------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+--------+
|  count|           5749132|           5749132|           5748125|            103698|           5749132|               2464|           5749132|            5748337|            5748337|            5748337|            5736289| 5749132|
|   mean| 33324.48559643438| 66587.43558331935|0.7129024704436274|0.

In [15]:
summary.select('summary', 'cmp_fname_c1', 'cmp_fname_c2').show()

+-------+------------------+------------------+
|summary|      cmp_fname_c1|      cmp_fname_c2|
+-------+------------------+------------------+
|  count|           5748125|            103698|
|   mean|0.7129024704436274|0.9000176718903216|
| stddev|0.3887583596162788|0.2713176105782331|
|    min|                 0|                 0|
|    max|                 1|                 1|
+-------+------------------+------------------+



In [16]:
matches = parsed.where('is_match = true')
match_summary = matches.describe()

misses = parsed.filter(col('is_match') == False)
miss_summary = misses.describe()

In [17]:
summary_p = summary.toPandas()

In [18]:
summary_p.head()

,summary,id_1,id_2,cmp_fname_c1,cmp_fname_c2,cmp_lname_c1,cmp_lname_c2,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz,is_match
0,count,5749132,5749132,5748125,103698,5749132,2464,5749132,5748337,5748337,5748337,5736289,5749132
1,mean,33324.48559643438,66587.43558331935,0.7129024704436274,0.9000176718903216,0.3156278193084133,0.31841283153174377,0.955001381078048,0.22446526708507172,0.48885529849763504,0.2227485966810923,0.00552866147434343,None
2,stddev,23659.859374488213,23620.487613269885,0.3887583596162788,0.2713176105782331,0.3342336339615816,0.36856706620066537,0.2073011111689795,0.4172297223846255,0.4998758236779038,0.41609096298317344,0.07414914925420066,None
3,min,1,10000,0,0,0,0,0,0,0,0,0,FALSE
4,max,9999,99999,1,1,1,1,1,1,1,1,1,TRUE


In [20]:
summary_p.shape

(5, 13)

In [21]:
summary_p = summary_p.set_index('summary').transpose().reset_index()

In [23]:
summary_p = summary_p.rename(columns={'index': 'field'})

In [25]:
summary_p = summary_p.rename_axis(None, axis=1)

In [27]:
summary_p.shape 

(12, 6)

In [28]:
summaryT = spark.createDataFrame(summary_p)

In [29]:
summaryT.show()

+------------+-------+-------------------+-------------------+-----+-----+
|       field|  count|               mean|             stddev|  min|  max|
+------------+-------+-------------------+-------------------+-----+-----+
|        id_1|5749132|  33324.48559643438| 23659.859374488213|    1| 9999|
|        id_2|5749132|  66587.43558331935| 23620.487613269885|10000|99999|
|cmp_fname_c1|5748125| 0.7129024704436274| 0.3887583596162788|    0|    1|
|cmp_fname_c2| 103698| 0.9000176718903216| 0.2713176105782331|    0|    1|
|cmp_lname_c1|5749132| 0.3156278193084133| 0.3342336339615816|    0|    1|
|cmp_lname_c2|   2464|0.31841283153174377|0.36856706620066537|    0|    1|
|     cmp_sex|5749132|  0.955001381078048| 0.2073011111689795|    0|    1|
|      cmp_bd|5748337|0.22446526708507172| 0.4172297223846255|    0|    1|
|      cmp_bm|5748337|0.48885529849763504| 0.4998758236779038|    0|    1|
|      cmp_by|5748337| 0.2227485966810923|0.41609096298317344|    0|    1|
|     cmp_plz|5736289|0.0

In [30]:
summaryT.printSchema()

root
 |-- field: string (nullable = true)
 |-- count: string (nullable = true)
 |-- mean: string (nullable = true)
 |-- stddev: string (nullable = true)
 |-- min: string (nullable = true)
 |-- max: string (nullable = true)



In [33]:
from pyspark.sql.types import DoubleType
for c in summaryT.columns:
    if c == 'field':
        continue
    summaryT = summaryT.withColumn(c, summaryT[c].cast(DoubleType()))

In [34]:
summaryT.printSchema()

root
 |-- field: string (nullable = true)
 |-- count: double (nullable = true)
 |-- mean: double (nullable = true)
 |-- stddev: double (nullable = true)
 |-- min: double (nullable = true)
 |-- max: double (nullable = true)



In [35]:
from pyspark.sql import DataFrame
from pyspark.sql.types import DoubleType

def pivot_summary(desc):
    desc_p = desc.toPandas()
    desc_p = desc_p.set_index('summary').transpose().reset_index()
    desc_p = desc_p.rename(columns={'index': 'field'})
    desc_p = desc_p.rename_axis(None, axis=1)
    descT = spark.createDataFrame(desc_p)
    for c in descT.columns:
        if c == 'field':
            continue
        else:
            descT = descT.withColumn(c, descT[c].cast(DoubleType()))
    return descT                            

In [37]:
match_summaryT = pivot_summary(match_summary)
miss_summaryT = pivot_summary(miss_summary)

In [38]:
match_summaryT.show()

+------------+-------+------------------+--------------------+-------+-------+
|       field|  count|              mean|              stddev|    min|    max|
+------------+-------+------------------+--------------------+-------+-------+
|        id_1|20931.0| 34575.72117911232|  21950.312851969127|10001.0|99946.0|
|        id_2|20931.0| 51259.95939037791|    24345.7334537752|10010.0|99996.0|
|cmp_fname_c1|20922.0|0.9973163859635038|0.036506675848336785|    0.0|    1.0|
|cmp_fname_c2| 1333.0|0.9898900320318174| 0.08251973727615237|    0.0|    1.0|
|cmp_lname_c1|20931.0|0.9970152595958817| 0.04311880753394512|    0.0|    1.0|
|cmp_lname_c2|  475.0| 0.969370167843852| 0.15345280740388917|    0.0|    1.0|
|     cmp_sex|20931.0| 0.987291577086618| 0.11201570591216432|    0.0|    1.0|
|      cmp_bd|20925.0|0.9970848267622461| 0.05391487659807977|    0.0|    1.0|
|      cmp_bm|20925.0|0.9979450418160095| 0.04528612745217063|    0.0|    1.0|
|      cmp_by|20925.0|0.9961290322580645|0.062098048

In [39]:
match_summaryT.createOrReplaceTempView('match_desc')
miss_summaryT.createOrReplaceTempView('miss_desc')

In [42]:
spark.sql("""
    SELECT a.field,
           a.count + b.count AS total,
           a.mean - b.mean AS delta
    FROM match_desc AS a
    INNER JOIN miss_desc AS b ON a.field = b.field
    WHERE a.field NOT IN ('id_1', 'id_2')
    ORDER BY delta DESC, total DESC

""").show()

+------------+---------+--------------------+
|       field|    total|               delta|
+------------+---------+--------------------+
|     cmp_plz|5736289.0|  0.9563812499852176|
|cmp_lname_c2|   2464.0|  0.8064147192926264|
|      cmp_by|5748337.0|  0.7762059675300512|
|      cmp_bd|5748337.0|   0.775442311783404|
|cmp_lname_c1|5749132.0|  0.6838772482590526|
|      cmp_bm|5748337.0|  0.5109496938298685|
|cmp_fname_c1|5748125.0|  0.2854529057460786|
|cmp_fname_c2| 103698.0| 0.09104268062280008|
|     cmp_sex|5749132.0|0.032408185250332844|
|    is_match|5749132.0|                NULL|
+------------+---------+--------------------+



In [44]:
good_features = ['cmp_lname_c1', 'cmp_plz', 'cmp_by', 'cmp_bd', 'cmp_bm']

In [47]:
sum_expression = ' + '.join(good_features)

In [48]:
sum_expression

'cmp_lname_c1 + cmp_plz + cmp_by + cmp_bd + cmp_bm'

In [49]:
from pyspark.sql.functions import expr

In [50]:
scored = parsed.fillna(0, subset=good_features).\
                withColumn('score', expr(sum_expression)).\
                select('score', 'is_match')

In [51]:
scored.show()

+-----+--------+
|score|is_match|
+-----+--------+
|  5.0|    TRUE|
|  5.0|    TRUE|
|  5.0|    TRUE|
|  5.0|    TRUE|
|  5.0|    TRUE|
|  5.0|    TRUE|
|  4.0|    TRUE|
|  5.0|    TRUE|
|  5.0|    TRUE|
|  5.0|    TRUE|
|  5.0|    TRUE|
|  5.0|    TRUE|
|  5.0|    TRUE|
|  5.0|    TRUE|
|  5.0|    TRUE|
|  5.0|    TRUE|
|  4.0|    TRUE|
|  5.0|    TRUE|
|  5.0|    TRUE|
|  5.0|    TRUE|
+-----+--------+
only showing top 20 rows



In [72]:
def crossTabs(scored: DataFrame, t: DoubleType):
    return scored.selectExpr(f'score >= {t} as above', 'is_match').\
           groupBy('above').pivot('is_match', ('true', 'false')).count()

In [73]:
scored.printSchema()

root
 |-- score: double (nullable = true)
 |-- is_match: string (nullable = true)



In [74]:
crossTabs(scored, 2.0).show()

+-----+----+-----+
|above|true|false|
+-----+----+-----+
| NULL|NULL| NULL|
| true|NULL| NULL|
|false|NULL| NULL|
+-----+----+-----+

